(1) Register on github.com in case you have not done this already.

Done!

(2) Initialize a new public repository for this assignment on GitHub.

Connected to the public repository Grossepieper_Nils_Assignment05

(3) For the following exercises of this assignment, follow the standard Git workflow (i.e., pull the latest version of the project to your local computer, then stage, commit, and push all the modifications that you make throughout the project). Every logical programming step should be well documented on GitHub with a meaningful commit message, so that other people (e.g., your course instructor) can follow and understand the development history. You can do this either using Shell commands or a Git GUI of your choice.

Future changes in this script will be commented on git.

(4) In the HTML file that you submit, include the hyperlink to the project repository (e.g., https://github.com/yourUserName/yourProjectName)

https://github.com/NilsGrossepieper/Grossepieper_Nils_Assignment05.git

(5) Visit the documentation website for the API provided by ticketmaster.com (see here). Familiarize yourself with the features and functionalities of the Ticketmaster Discovery API. Have a particular look at rate limits.

I have had a look at the website ticketmaster.com.
I found the following rate limits:
Limit: 5000 API calls per day
Rate limit: 5 requests per second.
Deep Paging: supports retrieving the 100th item (size * page < 1000)

(6) Whithin the scope of this assignment, you do not have to request your own API key. Instead retrieve a valid key from the API Explorer. This API key enables you to perform the GET requests needed throughout this assignment. Even though this API key is not secret per se (it is publicly visible on the API Explorer website), please comply to the common secrecy practices discussed in the lecture and the tutorial: Treat the API key as a secret token. Your API key should neither appear in the code that you are submitting nor in your public GitHub repository.